# Imports

In [1]:
import pandas as pd # pip install pandas
from difflib import SequenceMatcher
from dateutil.parser import parse

# Helper Functions

In [2]:
def levenshtein_distance(s1, s2):
    return SequenceMatcher(None, s1, s2).ratio()

def convert_date_column(df, column_name):
    df[column_name] = df[column_name].apply(lambda x: parse(x, dayfirst=True))
    df[column_name] = df[column_name].dt.strftime('%d-%m-%Y')
    df[column_name] = pd.to_datetime(df[column_name], format='%d-%m-%Y')

# Load Data

## Original Data Sources

* Football Matches of Spanish League
https://www.kaggle.com/datasets/ricardomoya/football-matches-of-spanish-league

* Spanish League 1995-2023
https://www.kaggle.com/datasets/kishan305/la-liga-results-19952020?select=LaLiga_Matches.csv

* Premier League Matches 1993-2023
https://www.kaggle.com/datasets/evangower/premier-league-matches-19922022

* Turkish Super League Matches (1959-2021)
https://www.kaggle.com/datasets/faruky/turkish-super-league-matches-19592020

* France Football from 1950 to 2022
https://www.kaggle.com/datasets/simonherv/france-football-from-1950-to-2022


* Football | Bundesliga Seasons 2005/06 - 2022/23
https://www.kaggle.com/datasets/oles04/bundesliga-seasons

* Bundesliga Results 1993-2018
https://www.kaggle.com/datasets/thefc17/bundesliga-results-19932018

## Import from Github

In [3]:
Bundesliga_1993 = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/Bundesliga%201993-2018.csv')
Bundesliga_2005 = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/Bundesliga%202005-2023.csv')
France_League = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/Ligue%201%201950-2022.csv')
Turkish_League = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/Turkish%20Super%20League%201959-2021.csv')
Premier_League = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/Premier%20League%201993-2023.csv')
Spanish_League = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/La%20Liga%201970-2017.csv')
Spanish_League_1995 = pd.read_csv('https://raw.githubusercontent.com/yasser-sulaiman/info-Int/main/data/La%20Liga%201995-2023.csv')

## Import Locally Jupyter

In [4]:
# Spanish_League = pd.read_csv('data/La Liga 1970-2017.csv')
# Spanish_League_1995 = pd.read_csv('data/La Liga 1995-2023.csv')
# Premier_League = pd.read_csv('data/Premier League 1993-2023.csv')
# Turkish_League = pd.read_csv('data/Turkish Super League 1959-2021.csv')
# France_League = pd.read_csv('data/Ligue 1 1950-2022.csv')
# Bundesliga_2005 = pd.read_csv('data/Bundesliga 2005-2023.csv')
# Bundesliga_1993 = pd.read_csv('data/Bundesliga 1993-2018.csv')

# Global Schema

* We need to define a global schema for all data sources  
* we add the "League" column to all datasets  
* we add the winner column to the datasets that does not already have it
* We apply some standardization and cleaning on the individual data sources if needed
* Our Suggested Global schema: ["Date", "HomeTeam", "AwayTeam", "HomeGoals", "AwayGoals", "Winner", "League", "Year"]  
* we match the columns of the sources with the best match from the global schema using Levenshtein distance and the data type of the column
* Finally we apply some standardization and cleaning on the integrated data


## Bundesliga_1993

In [5]:
Bundesliga_1993.columns

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Season'],
      dtype='object')

In [6]:
# Adding a 'League' column with the value 'Bundesliga' for each row to indicate the league name
Bundesliga_1993['League'] = 'Bundesliga'
if "Winner" not in Bundesliga_1993.columns:
    Bundesliga_1993['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(Bundesliga_1993['FTHG'], Bundesliga_1993['FTAG'])]

convert_date_column(Bundesliga_1993, 'Date')

Bundesliga_1993.head(2)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Season,League,Winner
0,D1,1993-08-07,Bayern Munich,Freiburg,3,1,H,NaN,NaN,NaN,1993-94,Bundesliga,H
1,D1,1993-08-07,Dortmund,Karlsruhe,2,1,H,NaN,NaN,NaN,1993-94,Bundesliga,H


## Bundesliga_2005

In [7]:
Bundesliga_2005.columns

Index(['Unnamed: 0', 'MATCH_DATE', 'LEAGUE_NAME', 'SEASON', 'LEAGUE',
       'FINISHED', 'LOCATION', 'VIEWER', 'MATCHDAY', 'MATCHDAY_NR',
       'HOME_TEAM_ID', 'HOME_TEAM_NAME', 'HOME_TEAM', 'HOME_ICON',
       'AWAY_TEAM_ID', 'AWAY_TEAM_NAME', 'AWAY_TEAM', 'AWAY_ICON',
       'GOALS_HOME', 'GOALS_AWAY', 'DRAW', 'WIN_HOME', 'WIN_AWAY'],
      dtype='object')

In [8]:
# Adding a 'League' column with the value 'Bundesliga' for each row to indicate the league name
Bundesliga_2005 = Bundesliga_2005.drop("LEAGUE", axis=1)  
Bundesliga_2005['League'] = 'Bundesliga'
if "Winner" not in Bundesliga_2005.columns:
    Bundesliga_2005['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(Bundesliga_2005['GOALS_HOME'], Bundesliga_2005['GOALS_AWAY'])]

convert_date_column(Bundesliga_2005, 'MATCH_DATE')

Bundesliga_2005.head(2)

,Unnamed: 0,MATCH_DATE,LEAGUE_NAME,SEASON,FINISHED,LOCATION,VIEWER,MATCHDAY,MATCHDAY_NR,HOME_TEAM_ID,...,AWAY_TEAM_NAME,AWAY_TEAM,AWAY_ICON,GOALS_HOME,GOALS_AWAY,DRAW,WIN_HOME,WIN_AWAY,League,Winner
0,0,2005-05-08,1. Fussball-Bundesliga 2005/2006,2005,True,München,NaN,1. Spieltag,1,40,...,Borussia Mönchengladbach,Gladbach,https://i.imgur.com/KSIk0Eu.png,3,0,0.0,1.0,0.0,Bundesliga,H
1,1,2005-06-08,1. Fussball-Bundesliga 2005/2006,2005,True,Köln,NaN,1. Spieltag,1,65,...,1. FSV Mainz 05,Mainz,https://upload.wikimedia.org/wikipedia/commons...,1,0,0.0,1.0,0.0,Bundesliga,H


### Matching Team Names in Bundesliga Datasets

In [9]:
# find the best match for each team in Bundesliga_1993
teams_2005 = [team for team in Bundesliga_2005['HOME_TEAM'].unique() if len(team) > 1]
for t1 in Bundesliga_1993['HomeTeam'].unique():
    best_match = ''
    best_match_score = 0
    for t2 in teams_2005:
        score = levenshtein_distance(t1.lower(), t2.lower())
        if score > best_match_score:
            best_match = t2
            best_match_score = score

    if best_match != '':
        # change each team name in Bundesliga_1993 to the best match from Bundesliga_2005
        Bundesliga_1993['HomeTeam'] = Bundesliga_1993['HomeTeam'].replace(t1, best_match)
        Bundesliga_1993['AwayTeam'] = Bundesliga_1993['AwayTeam'].replace(t1, best_match)

        # remove the best match from Bundesliga_2005 to avoid matching it again
        teams_2005.remove(best_match)

In [10]:
Bundesliga_1993.head(10)

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Season,League,Winner
0,D1,1993-08-07,Bayern,Freiburg,3,1,H,NaN,NaN,NaN,1993-94,Bundesliga,H
1,D1,1993-08-07,Paderborn,Karlsruhe,2,1,H,NaN,NaN,NaN,1993-94,Bundesliga,H
2,D1,1993-08-07,Duisburg,Leverkusen,2,2,D,NaN,NaN,NaN,1993-94,Bundesliga,D
3,D1,1993-08-07,Köln,Kaiserslautern,0,2,A,NaN,NaN,NaN,1993-94,Bundesliga,A
4,D1,1993-08-07,BVB,Nürnberg,5,2,H,NaN,NaN,NaN,1993-94,Bundesliga,H
5,D1,1993-08-07,Leipzig,Darmstadt,3,3,D,NaN,NaN,NaN,1993-94,Bundesliga,D
6,D1,1993-08-07,Gladbach,Frankfurt,0,4,A,NaN,NaN,NaN,1993-94,Bundesliga,A
7,D1,1993-08-07,Braunschweig,Schalke,3,0,H,NaN,NaN,NaN,1993-94,Bundesliga,H
8,D1,1993-08-07,Bremen,Stuttgart,5,1,H,NaN,NaN,NaN,1993-94,Bundesliga,H
9,D1,1993-08-14,Darmstadt,Duisburg,0,1,A,NaN,NaN,NaN,1993-94,Bundesliga,A


In [11]:
german_teams = Bundesliga_1993['HomeTeam'].unique().tolist()

## France_League

In [12]:
France_League.columns

Index(['id', 'date', 'local', 'visiteur', 'ligue', 'saison', 'local_score',
       'visiteur_score', 'resultat'],
      dtype='object')

In [13]:
# Adding a 'League' column with the value 'Bundesliga' for each row to indicate the league name
France_League = France_League.drop("resultat", axis=1)
France_League['League'] = 'Ligue 1'
if "Winner" not in France_League.columns:
    France_League['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(France_League['local_score'], France_League['visiteur_score'])]

convert_date_column(France_League, 'date')

France_League.head(2)

,id,date,local,visiteur,ligue,saison,local_score,visiteur_score,League,Winner
0,110843,2023-01-29,OGC Nice,LOSC Lille,Ligue 1,2022-2023,1,0,Ligue 1,H
1,110844,2023-01-29,AJ Auxerre,Montpellier HSC,Ligue 1,2022-2023,0,2,Ligue 1,A


In [14]:
french_teams = France_League['local'].unique().tolist()

## Premier_League

In [15]:
Premier_League.columns

Index(['Season_End_Year', 'Wk', 'Date', 'Home', 'HomeGoals', 'AwayGoals',
       'Away', 'FTR'],
      dtype='object')

In [16]:
# Adding a 'League' column with the value 'Bundesliga' for each row to indicate the league name
Premier_League['League'] = 'Premier League'
if "Winner" not in Premier_League.columns:
    Premier_League['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(Premier_League['HomeGoals'], Premier_League['AwayGoals'])]

convert_date_column(Premier_League, 'Date')

Premier_League.head(2)

,Season_End_Year,Wk,Date,Home,HomeGoals,AwayGoals,Away,FTR,League,Winner
0,1993,1,1992-08-15,Coventry City,2,1,Middlesbrough,H,Premier League,H
1,1993,1,1992-08-15,Leeds United,2,1,Wimbledon,H,Premier League,H


In [17]:
english_teams = Premier_League['Home'].unique().tolist()

## Spanish_League

In [18]:
Spanish_League.columns

Index(['id', 'season', 'division', 'round', 'localTeam', 'visitorTeam',
       'localGoals', 'visitorGoals', 'date', 'timestamp'],
      dtype='object')

In [19]:
# Adding a 'League' column with the value 'Bundesliga' for each row to indicate the league name
Spanish_League['League'] = 'La Liga'
if "Winner" not in Spanish_League.columns:
    Spanish_League['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(Spanish_League['localGoals'], Spanish_League['visitorGoals'])]

convert_date_column(Spanish_League, 'date')

Spanish_League.head(2)

,id,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date,timestamp,League,Winner
0,1,1970-71,1,1,Atletico de Bilbao,Barcelona,1,1,1970-09-12,21938400,La Liga,D
1,2,1970-71,1,1,Las Palmas,Atletico de Madrid,1,1,1970-09-12,21938400,La Liga,D


## Spanish League 1995-2023

In [20]:
Spanish_League_1995.columns

Index(['Season', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR'],
      dtype='object')

In [21]:
# Adding a 'League' column with the value 'Bundesliga' for each row to indicate the league name
Spanish_League_1995['League'] = 'La Liga'
if "Winner" not in Spanish_League_1995.columns:
    Spanish_League_1995['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(Spanish_League_1995['FTHG'], Spanish_League_1995['FTAG'])]

convert_date_column(Spanish_League_1995, 'Date')

Spanish_League_1995.head(2)

,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,League,Winner
0,1995-96,1995-09-02,La Coruna,Valencia,3,0,H,2.0,0.0,H,La Liga,H
1,1995-96,1995-09-02,Sp Gijon,Albacete,3,0,H,3.0,0.0,H,La Liga,H


In [22]:
# find the best match for each team in Spanish_League
teams_1995 = [team for team in Spanish_League_1995['HomeTeam'].unique() if len(team) > 1]
for t1 in Spanish_League['localTeam'].unique():
    best_match = ''
    best_match_score = 0
    for t2 in teams_1995:
        score = levenshtein_distance(t1.lower(), t2.lower())
        if score > best_match_score:
            best_match = t2
            best_match_score = score

    if best_match != '':
        # change each team name in Spanish_League to the best match from Spanish_League_1995
        Spanish_League['localTeam'] = Spanish_League['localTeam'].replace(t1, best_match)
        Spanish_League['visitorTeam'] = Spanish_League['visitorTeam'].replace(t1, best_match)

        # remove the best match from teams_1995 to avoid matching it again
        teams_1995.remove(best_match)

In [23]:
Spanish_League

,id,season,division,round,localTeam,visitorTeam,localGoals,visitorGoals,date,timestamp,League,Winner
0,1,1970-71,1,1,Ath Bilbao,Barcelona,1,1,1970-09-12,21938400,La Liga,D
1,2,1970-71,1,1,Las Palmas,Ath Madrid,1,1,1970-09-12,21938400,La Liga,D
2,3,1970-71,1,1,Real Madrid,Valencia,2,0,1970-09-12,21938400,La Liga,H
3,4,1970-71,1,1,Celta,Sp Gijon,2,0,1970-09-13,22024800,La Liga,H
4,5,1970-71,1,1,Elche,Granada,1,1,1970-09-13,22024800,La Liga,D
...,...,...,...,...,...,...,...,...,...,...,...,...
37142,37143,2017-18,2,42,Alaves,Girona,2,0,2018-06-02,1527890400,La Liga,H
37143,37144,2017-18,2,42,Villareal,Huesca,2,1,2018-06-02,1527890400,La Liga,H
37144,37145,2017-18,2,42,Alcorcon,Reus,3,0,2018-06-02,1527890400,La Liga,H
37145,37146,2017-18,2,42,Numancia,Leganes,2,1,2018-06-02,1527890400,La Liga,H


In [24]:
spanish_teams = Spanish_League['localTeam'].unique().tolist()

## Turkish_League

In [25]:
Turkish_League.columns

Index(['Date', 'Season', 'Week', 'home', 'visitor', 'FT', 'hgoal', 'vgoal',
       'division', 'tier', 'totgoal', 'goaldiff', 'result', 'HT', 'hgoal_half',
       'vgoal_half', 'half_totgoal', 'half_goaldiff', 'result_half', 'fans',
       'neutral', 'home_red_card', 'visitor_red_card'],
      dtype='object')

In [26]:
# Adding a 'League' column with the value 'Bundesliga' for each row to indicate the league name
Turkish_League['League'] = 'Turkish Super League'
if "Winner" not in Turkish_League.columns:
    Turkish_League['Winner'] = ['H' if x > y else 'A' if x < y else 'D' for x, y in zip(Turkish_League['hgoal'], Turkish_League['vgoal'])]

convert_date_column(Turkish_League, 'Date')

Turkish_League.head(2)

,Date,Season,Week,home,visitor,FT,hgoal,vgoal,division,tier,...,vgoal_half,half_totgoal,half_goaldiff,result_half,fans,neutral,home_red_card,visitor_red_card,League,Winner
0,1959-02-21,1958,1,Genclerbirligi,Adalet,1-1,1,1,T1,1,...,1,1,-1,A,1,0,0,0,Turkish Super League,D
1,1959-02-21,1958,1,Izmirspor,Beykoz 1908,2-1,2,1,T1,1,...,0,2,2,H,1,0,0,0,Turkish Super League,H


In [27]:
turkish_teams = Turkish_League['home'].unique().tolist()

# Schema Matching Using Levenshtein Distance

In [28]:
global_schema = {
    "Winner":["object", "str"], 
    "League":["object", "str"],
    "Date":["datetime64[ns]", "date"], 
    "HomeTeam":["object", "str"], 
    "AwayTeam":["object", "str"], 
    "HomeGoals":[int, float, "int64"], 
    "AwayGoals":[int, float, "int64"]
    }

leagues_list = [Bundesliga_1993, Bundesliga_2005, France_League, Premier_League, Spanish_League, Spanish_League_1995, Turkish_League]
new_leagues_list = []

for league in leagues_list:
    cols = list(league.columns)
    matched_cols = []

    for global_col, global_col_dtype in global_schema.items():

        best_match = ''
        best_match_score = 0

        for col in cols:

            score = levenshtein_distance(global_col.lower(), col.lower())
            dtype_col = league[col].dtype
            
            if (score > best_match_score) and (dtype_col in global_col_dtype):
                best_match = col
                best_match_score = score

        if best_match != '':
            matched_cols.append(best_match)
            cols.remove(best_match)
            

    new_league = league.loc[:, matched_cols]
    new_col_names = dict(zip(matched_cols, global_schema.keys()))
    new_league = new_league.rename(columns=new_col_names)
    new_leagues_list.append(new_league)


# Data Integration

In [29]:
#concatenate dataframes in new leagues list
data = pd.concat(new_leagues_list, ignore_index=True)

#checks whether the length of the concatenated DataFrame data is equal to the sum of the lengths of all the individual DataFrames
assert len(data) == sum([len(l) for l in new_leagues_list])

In [30]:
# add year column for easier queries
data['Year'] = data.Date.dt.year

In [31]:
all_teams = german_teams + english_teams + french_teams + spanish_teams + turkish_teams
data = data[data['AwayTeam'].isin(all_teams)]
data = data[data['HomeTeam'].isin(all_teams)].reset_index(drop=True)

In [32]:
# check results are correct
data = data[(data['HomeGoals'] < data['AwayGoals']) & (data['Winner'] == 'A') |
            (data['HomeGoals'] > data['AwayGoals']) & (data['Winner'] == 'H') |
            (data['HomeGoals'] == data['AwayGoals']) & (data['Winner'] == 'D')]
data

,Winner,League,Date,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Year
0,H,Bundesliga,1993-08-07,Bayern,Freiburg,3,1,1993
1,H,Bundesliga,1993-08-07,Paderborn,Karlsruhe,2,1,1993
2,D,Bundesliga,1993-08-07,Duisburg,Leverkusen,2,2,1993
3,A,Bundesliga,1993-08-07,Köln,Kaiserslautern,0,2,1993
4,H,Bundesliga,1993-08-07,BVB,Nürnberg,5,2,1993
...,...,...,...,...,...,...,...,...
178398,A,La Liga,2023-09-30,Girona,Real Madrid,0,3,2023
178399,H,La Liga,2023-09-30,Sociedad,Ath Bilbao,3,0,2023
178400,D,La Liga,2023-10-01,Almeria,Granada,3,3,2023
178401,H,La Liga,2023-10-01,Ath Madrid,Cadiz,3,2,2023


# Remove Duplicates

In [33]:
# Checking for duplicates in the updated data
duplicates_check = data[data.duplicated(subset=["Date", "HomeTeam", "League", "Winner"], keep=False)]

# Printing the duplicates (if any)
if len(duplicates_check) == 0:
    print("No duplicates found")
else:
    print("There are {} duplicates".format(len(duplicates_check)))


# Remove duplicates 
data_no_duplicates = data.drop_duplicates(subset=["Date", "HomeTeam", "League", "Winner"], keep='first')

# Checking for duplicates in the updated data
duplicates_check = data_no_duplicates[data_no_duplicates.duplicated(subset=["Date", "HomeTeam", "League", "Winner"], keep=False)]

# Printing the duplicates (if any)
if len(duplicates_check) == 0:
    print("No duplicates found after removing duplicates")
else:
    print("Duplicates still exist after removing duplicates")
    print(duplicates_check)

There are 13950 duplicates
No duplicates found after removing duplicates


# Save Data to CVS File

In [115]:
file_path = 'data/auto_integrated_data.csv'  # Modify 'path_to_save' and file name as needed
data_no_duplicates.to_csv(file_path, index=False)